In [3]:
import pandas as pd
from sklearn.dummy import DummyClassifier

In [5]:
df = pd.read_csv("train.csv")

In [8]:
def ML_pipeline_kfold2(X,y,rs,n_folds,ML_algo,param_grid):
    # create a test set
    X_other, X_test, y_other, y_test = train_test_split(X,y,test_size = 0.2, stratify = X['MTRANS'],random_state=rs)
    # splitter for _other
    kf = StratifiedKFold(n_splits=n_folds,shuffle=True,random_state=rs)
    # create the pipeline: preprocessor + supervised ML method
    cat_ftrs = ['Gender','family_history_with_overweight','FAVC','SMOKE','SCC','MTRANS']
    ordinal_ftrs = ['CAEC','CALC']
    ordinal_cats = [['no','Sometimes','Frequently','Always'],['no','Sometimes','Frequently','Always']]
    num_ftrs = ['Age','Height','Weight','FCVC','NCP','CH2O','FAF','TUE']
    # one-hot encoder
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(sparse=False,handle_unknown='ignore'))])

    # ordinal encoder
    ordinal_transformer = Pipeline(steps=[
        ('ordinal', OrdinalEncoder(categories = ordinal_cats))])

    # standard scaler
    numeric_transformer = Pipeline(steps=[
        ('scaler', StandardScaler())])

    # collect the transformers
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, num_ftrs),
            ('cat', categorical_transformer, cat_ftrs),
            ('ord', ordinal_transformer, ordinal_ftrs)])
    le = LabelEncoder()
    X_other_prep = preprocessor.fit_transform(X_other)
    X_test_prep = preprocessor.transform(X_test)
    y_other_prep = le.fit_transform(y_other)
    y_test_prep = le.transform(y_test)
    pipe = make_pipeline(ML_algo)
    print(pipe.get_params().keys())
#     df_train = pd.DataFrame(data=X_prep,columns=feature_names)
#     print(df_train.shape)

#     # transform the CV
#     df_CV = preprocessor.transform(X_CV)
#     df_CV = pd.DataFrame(data=df_CV,columns = feature_names)
#     print(df_CV.shape)

#     # transform the test
#     df_test = preprocessor.transform(X_test)
#     df_test = pd.DataFrame(data=df_test,columns = feature_names)
#     print(df_test.shape)
    # prepare gridsearch
    grid = GridSearchCV(pipe, param_grid=param_grid,cv=kf, return_train_score = True,n_jobs=-1,verbose=10)
    # do kfold CV on _other
    grid.fit(X_other_prep, y_other_prep)
    feature_names = num_ftrs + ordinal_ftrs+ \
                list(preprocessor.named_transformers_['cat'][0].get_feature_names(cat_ftrs))
    y_test_pred = grid.predict(X_test_prep)
    X_other2 = pd.DataFrame(data=X_other_prep,columns = feature_names)
    X_test2 = pd.DataFrame(data=X_test_prep,columns = feature_names)
    return grid, np.array(feature_names),X_other2, y_other_prep, X_test2, y_test_prep, y_test_pred, preprocessor, pipe, le

In [11]:
from sklearn.dummy import DummyClassifier
ML_algo = DummyClassifier(strategy = "most_frequent")
param_grid = {
             }
X = df['image_id']
y = df['label']

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, y)
dummy_clf.score(X, y)

0.6149460204701593